## Definição

Você consegue visualizar agora que todos os componentes LangChain apresentados, podem ser encadeados, uma vez que eles implementam a interface `Runnable`, ou seja, podemos montar nossas cadeias (chains) e, vale lembrar que a saída de um componente será a entrada do próximo componente.

Chain portanto é a cadeia formado por elementos de LangChain que implementam uma determinada ação, principalmente envolvendo a atuação de LLMs. Ou seja, a chain nada mais é do que amarrar uma série de tarefas realizadas por cada componente em um único fluxo linear.

Para esta aula preparei algo mais prático, ou seja, vamos praticar diferentes formas de encadeamento de componentes, por isso é fundamental você ter visto as aulas anteriores e entender também os `runnables` já que vamos utilizá-los em nossos exercícios.

## Exemplo - Básico

Vamos criar uma chain que é a mais simples envolvendo um prompt, um modelo e um analisador de saída. Já vimos em aulas anteriores, mas vale relembrar e é um exemplo bom para começarmos até aprofundar nas cadeias mais complexas.

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from helpers.llm import initialize_llm, logger, pretty_print
from langchain.prompts import ChatPromptTemplate  
from langchain.schema.output_parser import StrOutputParser  
from langchain_core.runnables import RunnableLambda  

llm, _, embeddings = initialize_llm()

In [ ]:

# Definindo o prompt de comunicação - adotamos aqui um estilo chat prompt  
# template, uma vez que estamos usando o modelo do tipo chat.  
  
prompt_sistema = """Você é um assistente especialista em criar conteúdo para o twitter e tem como objetivo   
criar os melhores tweets virais sobre o tema que o usuário te passar. Seja criativo e atenda ao padrão de 280 caracteres do   
twitter.  
"""  
  
prompt_template = ChatPromptTemplate(  
    [  
        ("system", prompt_sistema),  
        ("human", "Crie um total de {numero_de_publicacoes} tweets sobre o tema {input_tema}."),  
    ]  
)  
  
# Crie a cadeia combinada usando LangChain Expression Language (LCEL)  
chain = prompt_template | llm | StrOutputParser()  
  
  
# Executamos nossa chain  
result = chain.invoke({"numero_de_publicacoes": 3, "input_tema": "tecnologia"})  
  
# Imprimimos a saída.  
print(result)

## Exemplo - Chains sequenciais com funções personalizadas

Vamos criar uma chain agora envolvendo a criação de funções personalizadas, ou seja, acoplaremos aqui a atuação dos `RunnableLambda` para que possamos converter uma função personalizada em componente LangChain e ser possivel adicionar nossa função em meio à cadeia.

Digamos que nós queremos confirmar se o total de caracteres de cada tweet está dentro dos 280 caracteres permitido. Vamos aproveitar o nosso exemplo anterior sobre a criação de tweets para gerar um relatório de validação.

1. Vamos primeiro ajustar nosso prompt para que o LLM sempre gere o total de tweets separados por uma quebra de linha (\n).
2. Em seguida, vamos construir uma função que funcionará como um analisador de saida personalizado ou seja, pegará a saída `string` do modelo e irá gerar uma lista em que cada elemento dessa lista será um tweet criado pelo LLM. Aqui usaremos o `RunnableLambda` para nos ajudar.
3. Depois teremos uma outra função personalizada que analisa essa lista e conta o total de caracteres de cada tweet. O retorno dela será um dicionário com os resultado. Aqui usaremos o `RunnableLambda` para nos ajudar também.

In [ ]:
# Criando uma função personalizada para tratar a saida textual do LLM  
  
def separador_de_tweet(entrada: str) -> list:  
    """  
    Função que recebe uma string e retorna uma lista com os elementos separados por quebras de linha.  
    Args:        entrada (str): A string de entrada, onde os valores estão separados por quebras de linha.  
    Returns:        list: Uma lista contendo cada elemento da string como um item separado.    """    # Divide a string em uma lista utilizando o caractere de quebra de linha '\n'  
    elementos = entrada.split('\n')  
  
    # Remove espaços extras e ignora linhas vazias  
    elementos_limpos = [elemento.strip() for elemento in elementos if elemento.strip()]  
  
    return elementos_limpos  
  
# Criando uma função personalizada pegar a lista criada na função anterior e gerar um dicionário com o relátorio de  
# analise de caracteres.  
  
def relatorio_de_analise_de_caracteres(entrada: list) -> dict:  
    """  
    Função que gera um relatório com os tweets e a contagem de caracteres de cada tweet.  
    Args:        entrada (list): Lista de strings representando os tweets.  
    Returns:        dict: Um dicionário com duas chaves:              - 'tweets': contendo a lista original.              - 'num_caract': contendo uma lista com o número de caracteres de cada tweet.    """    # Gera a contagem de caracteres para cada item na lista  
    contagem_caracteres = [len(tweet) for tweet in entrada]  
  
    # Monta o dicionário de saída  
    relatorio = {  
        'tweets': entrada,  
        'num_caract': contagem_caracteres  
    }  
  
    return relatorio  

In [ ]:
# ------------------------------------------------------------------------------
  
# Definindo o prompt de comunicação - adotamos aqui um estilo chat prompt  
# template, uma vez que estamos usando o modelo do tipo chat.  
  
prompt_sistema = """Você é um assistente especialista em criar conteúdo para o twitter e tem como objetivo  
criar os melhores tweets virais sobre o tema que o usuário te passar. Seja criativo e atenda ao padrão de 280 caracteres do   
twitter.  
Orientação:  
- Crie apenas o numero de tweets informado.  
- Separe cada um deles por uma quebra de linha,  
"""  
  
prompt_template = ChatPromptTemplate(  
    [  
        ("system", prompt_sistema),  
        ("human", "Crie um total de {numero_de_publicacoes} tweets sobre o tema {input_tema}."),  
    ])

In [ ]:
# Crie a cadeia combinada usando LangChain Expression Language (LCEL).  
# para adicionar os outros componentes personalizados à cadeia, precisamos converter as funções em um componente langchain, para isso  
# precisamos usar o RunnableLambda.  
  
chain = prompt_template | llm | StrOutputParser() | RunnableLambda(separador_de_tweet) | RunnableLambda(relatorio_de_analise_de_caracteres)  

In [ ]:
# Executamos nossa chain  
result = chain.invoke({"numero_de_publicacoes": 3, "input_tema": "tecnologia"})  
  
# Imprimimos o nosso dicionário de relatório:  
print(result)  
print("-"*50)  
  
# imprimindo de forma mais estruturada:  
for i, (tweet, num_caract) in enumerate(zip(result['tweets'], result['num_caract']), start=1):  
        print(f"Tweet {i}: {tweet}")  
        print(f"Total de caracteres: {num_caract}")  
        if num_caract <= 280:  
            print("Validação: OK")  
        else:  
            print("Validação: Tweet supera o limite de 280 caracteres")  
        print("-"*50)

## Exemplo - Chains de execução paralela

Agora vamos a um problema mais complexo, imagine que queremos criar um assistente que recebe uma review de um filme e ele tem que analisar prós e contras comentados no review entregue na entrada.

Para que possamos entender a execução paralela de chain, vamos criar uma arquitetura onde a análise pró e contra são realizadas de forma paralela, e no final vamos unir os resultados.

In [ ]:
# Definindo o prompt de comunicação - adotamos aqui um estilo chat prompt  
# template, uma vez que estamos usando o modelo do tipo chat.  
  
prompt_template = ChatPromptTemplate.from_messages(  
    [  
        ("system", "Você é um escritor especialista em análises de review de filmes de cinema."),  
        ("human", "Liste de forma estruturada os principais detalhes e pontos de vistas apresentados na seguinte  review entregue pelo usuário não invente nada apenas capture as principais informações apresentadas. Review: {movie_review}."),  
    ]  
)  

  
# Vamos definir um braço da nossa chain que será uma cadeia intermediária de analise dos pontos positivos sobre a review.  
  
analise_ponto_positivo_template = ChatPromptTemplate(  
    [  
        ("system", "Você é um analista crítico de filmes de cinema"),  
        (  
            "human", "Dados este review estruturado: {review_estruturado}, liste os pontos positivos do filme.",  
        ),  
    ]  
)  

# criando a chain do braço 1  
chain_intermediaria_positiva = analise_ponto_positivo_template | llm | StrOutputParser()  
  
# -------------------------------------------------------------------------------- 
  
# Vamos definir outro braço da nossa chain que será uma cadeia intermediária de analise dos pontos negativos sobre a review.  
  
analise_ponto_negativo_template = ChatPromptTemplate(  
    [  
        ("system", "Você é um analista crítico de filmes de cinema"),  
        (  
            "human", "Dados este review estruturado: {review_estruturado}, liste os pontos negativos do filme.",  
        ),  
    ]  
)  
  
# criando a chain do braço 2  
chain_intermediaria_negativa = analise_ponto_negativo_template | llm | StrOutputParser()  
  
# -------------------------------------------------------------------------------- 
  
# Função responsável por combinar os resultados dos braços que vão ser executados em paralelo.  
def combinando_analises(entrada: dict):  
    return f"Análise positiva:\n{entrada['posivita']}\n\nAnálise negativa:\n{entrada['negativa']}"  

In [ ]:
# Crie a cadeia combinada usando LangChain Expression Language (LCEL)  
# Em RunnableLambda(lambda x: {"review_estruturado": x}) estamos convertendo a saida string para um dicionário  
# com a chave 'review_estruturado' que os templates das chains intermediárias exige como entrada.  
  
chain = (prompt_template  
         | llm  
         | StrOutputParser()  
         | RunnableLambda(lambda x: {"review_estruturado": x})  
         | {"posivita": chain_intermediaria_positiva, "negativa": chain_intermediaria_negativa}  
         | RunnableLambda(combinando_analises)  
         )  

In [ ]:
# Executando nossa chain principal.  
movie_review ="""Crítica de "O Gladiador 2"  
"O Gladiador 2", dirigido por Ridley Scott, chega aos cinemas com a expectativa de reviver a grandiosidade épica de   
seu antecessor. No entanto, apesar do histórico impressionante de Scott, que inclui clássicos como "Blade Runner" e   
"Alien", o filme parece tropeçar em sua própria ambição.  
  
Desde o início, o filme tenta inovar ao incorporar cenas animadas e elementos de inteligência artificial,   
possivelmente como uma homenagem ao primeiro "Gladiador". No entanto, essa escolha estética, embora ousada,   
não se integra de maneira fluida à narrativa, criando uma desconexão que pode confundir o espectador.  
  
A tentativa de trazer novidade às lutas no Coliseu, com a introdução de navios vikings e tubarões, é um exemplo   
de como o filme busca surpreender. No entanto, essas cenas acabam por sacrificar a autenticidade histórica em prol   
do espetáculo, o que pode afastar aqueles que esperavam uma representação mais fiel das arenas romanas. A inclusão   
de macacos em combate, por sua vez, remete a outras franquias cinematográficas, diluindo ainda mais a originalidade   
do enredo.  
  
Apesar dessas escolhas questionáveis, é importante reconhecer o esforço de Scott em tentar oferecer algo novo e   
visualmente impactante. No entanto, a falta de uma pesquisa histórica mais aprofundada se faz sentir, e o filme   
poderia ter se beneficiado de uma abordagem mais cuidadosa nesse aspecto.  
  
Em suma, "O Gladiador 2" é uma obra que, embora repleta de potencial e com momentos de brilho visual, acaba por se   
perder em sua tentativa de inovar. Para os fãs do gênero e do diretor, pode ser uma experiência mista, que levanta   
questões sobre até que ponto a inovação deve ir sem comprometer a essência e a coerência da narrativa.  
"""  
  
result = chain.invoke({"movie_review": movie_review})  
  
# --------------------------------------------------------------------------------
# Imprimindo a saida.  
print(result)

## Exemplo - Chain de roteamento (Branch)

Agora vamos a um problema onde teremos uma função de roteamento. Essa função terá a função de decidir, usando a classificação por meio de um LLM, para qual branch (braço/ramo) da nossa chain o programa deve seguir e finalizar.

Imagine um sistema de atendimento com duas rotas: (1) quando o usuário solicita que seja atendido por um  humano, encaminhamos a pergunta para um atendente humano e finalizamos a cadeia ou (2) quando o usuário apenas deseja tirar dúvidas sobre um produto, nosso sistema escolhe enviar a pergunta para uma chain que implementa um bot capaz de responder às dúvidas desse usuário e finaliza a iteração.

A peça principal desse nosso sistema está na criação de uma função que fará esse roteamento entre cadeias de LangChain. No nosso caso, você pode observar isso na função 'executa_roteamento' implementada:

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser  
from pydantic import BaseModel, Field 

## Definindo a estrutura da chain que vai avaliar a entrada e retornar uma classificação para nossa função 'executa_roteamento'  
# Definindo a minha estrutura de saída usando Pydantic  
class Rota(BaseModel):  
    opcao: bool = Field(description="Defina True se necessitar atendimento humano e false caso contrário.")  
    pergunta_user: str = Field(description="Colocar neste parametro a pergunta do usuário sem alterá-la.")  
  
  
parser = PydanticOutputParser(pydantic_object=Rota)  
  
sys_prompt_rota = """Você é um especialista em classificação. Você receberá perguntas do usuário e precisará classificar, 
de forma booleana, se o usuário está solicitando conversar com um atendente humano ou não.  
\n{format_instructions}\n  
Pergunta Usuário: {pergunta_user}"  
"""  
  
rota_prompt_template = ChatPromptTemplate([("system", sys_prompt_rota),],  
                                          partial_variables={"format_instructions": parser.get_format_instructions()}  
                                          )  
  
# criando o pedaço da chain que controla o roteamento entre as branches  
chain_de_roteamento = rota_prompt_template | llm | parser  
  
# Se quiser testar a cadeia intermediária de roteamento:  
# result = chain_de_roteamento.invoke({"pergunta_user": "Quero falar com um humano"})  
  

In [ ]:
# Definindo o prompt de chatbot que tira duvidas do usuário:  
  
sys_chatbot_prompt = """ Você é um assistente de uma clinica odontológica e tem como objetivo responder à perguntas dos clientes. A seguir você  
encontra a FAQ do nosso site, use essas informações para realizar o atendimento e tirar dúvidas. Caso você desconheça alguma  
informação, não invente. Seja sempre amigável e esteja disposto a ajudar!  
  
**FAQ - Clínica Odontológica**  
1. **Quais serviços a clínica oferece?**    
   Oferecemos tratamentos como limpeza dental, clareamento, ortodontia, implantes, próteses, tratamento de canal e estética dental.  
2. **A clínica aceita convênios?**    
   Sim, trabalhamos com os principais convênios odontológicos. Consulte nossa equipe para verificar se aceitamos o seu.  
3. **Como agendar uma consulta?**    
   Você pode agendar sua consulta pelo telefone, WhatsApp ou diretamente em nosso site.  
4. **Quanto tempo dura uma consulta?**    
   Depende do procedimento, mas consultas de rotina geralmente duram entre 30 e 60 minutos.  
5. **Vocês atendem emergências?**    
   Sim, oferecemos atendimento emergencial para dores agudas, traumas ou casos de urgência.  
6. **É possível parcelar tratamentos?**    
   Sim, oferecemos opções de parcelamento. Entre em contato para conhecer os detalhes.  
7. **Crianças podem ser atendidas na clínica?**    
   Sim, contamos com profissionais especializados em odontopediatria para cuidar dos sorrisos dos pequenos.  
8. **O clareamento dental é seguro?**    
   Sim, nossos tratamentos de clareamento são realizados com técnicas e produtos seguros, supervisionados por especialistas.  
Se tiver mais dúvidas, entre em contato conosco! 😊  
  
Dúvida do usuário: {pergunta_user}  
"""  
  
prompt_template_chatbot = ChatPromptTemplate.from_messages([("system", sys_chatbot_prompt),])  
  
chain_chatbot = prompt_template_chatbot | llm | StrOutputParser()  
  

In [ ]:
## Definindo a função de escolha de roteamento (nó de rota)  
def executa_roteamento(entrada: Rota):  
    if entrada.opcao:  
        print(f"Opção classe Pydantic: {entrada.opcao} (Atendimento humano)")  
        return "Atendimento redirecionado para um humano. Favor aguardar alguns minutos que já vamos te atender!"  
    else:  
        print(f"Opção classe Pydantic: {entrada.opcao} (Atendimento Chatbot)")  
        return RunnableLambda(lambda x: {"pergunta_user": x.pergunta_user}) | chain_chatbot  


# Crie a cadeia final usando LangChain Expression Language (LCEL)  
chain = chain_de_roteamento | RunnableLambda(executa_roteamento) 

In [ ]:
# Executando nossa chain principal.  
result = chain.invoke({"pergunta_user": "Quais serviços a clínica oferece?"})  
  
# -------------------------------------------------------------------------------- 
# Imprimindo a saida.  
print("---------------")  
print(result)  
print("---------------")

#### Por que usar RouterChains?
A principal vantagem de usar RouterChains é a capacidade de criar aplicações mais robustas e flexíveis. Imagine que você está desenvolvendo um chatbot para uma loja online. Com RouterChains, você pode direcionar perguntas sobre produtos eletrônicos para um especialista em eletrônicos, enquanto perguntas sobre roupas são direcionadas para um especialista em moda. Isso melhora a precisão e a relevância das respostas, proporcionando uma melhor experiência ao usuário.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chain = (
    PromptTemplate.from_template(
        """
        Classifique a pergunta do usuário em uma das seguintes categorias:
        - Matemática
        - Física
        - Química
        - Outras Informações

        Pergunta: {query}
        Classificação:
        """
    )
    | llm
    | StrOutputParser() 
)
# Definindo os prompts especializados
chain_fisica = (
    PromptTemplate.from_template(
        """
        Você é um professor de física muito inteligente. Assuntos detalhados:
        - Mecânica: Estuda o movimento e as causas que o produzem ou alteram. Inclui cinemática (estudo do movimento sem considerar suas causas), dinâmica (estudo das forças que causam o movimento) e estática (estudo do equilíbrio de corpos). 
        - Termologia: Estuda o calor, a temperatura e suas relações. Abrange tópicos como calorimetria (estudo da troca de calor), termodinâmica (estudo das leis que regem as transformações de energia térmica), e estados físicos da matéria. 
        - Ondulatória: Estuda as ondas, fenômenos como a propagação de energia através de um meio ou do espaço. 
        - Óptica: Estuda a luz e seus fenômenos, como reflexão, refração, difração e interferência. 
        - Eletromagnetismo: Estuda os fenômenos relacionados à eletricidade e ao magnetismo, incluindo eletrostática (estudo das cargas elétricas em repouso), eletrodinâmica (estudo das cargas em movimento), e magnetismo. 
        - Pessoas influentes da area
        - Outras Informações

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)
chain_matematica = (
    PromptTemplate.from_template(
        """
        Você é um professor de matemática muito inteligente. Assuntos detalhados:
        - Álgebra: Estuda as operações e relações entre números, incluindo equações, funções e polinômios.
        - Geometria: Estuda as propriedades e relações de pontos, linhas, superfícies e sólidos no espaço.
        - Trigonometria: Estuda as relações entre os ângulos e os lados dos triângulos.
        - Cálculo: Estuda as taxas de variação e as somas infinitas, incluindo derivadas e integrais.
        - Estatística: Estuda a coleta, análise e interpretação de dados numéricos.
        - Pessoas influentes da area
        - Outras Informações

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)
chain_quimica = (
    PromptTemplate.from_template(
        """
        Você é um professor de química muito inteligente. Assuntos detalhados:
        - Estrutura Atômica: Estudo da composição do átomo, incluindo partículas subatômicas (prótons, nêutrons, elétrons), evolução dos modelos atômicos e distribuição eletrônica. 
        - Tabela Periódica: Organização dos elementos químicos, suas propriedades periódicas (eletroafinidade, energia de ionização, etc.) e relação com a estrutura atômica. 
        - Ligações Químicas: Tipos de ligações (iônica, covalente, metálica) e suas características, além de geometria molecular e polaridade. 
        - Funções Inorgânicas: Estudo de ácidos, bases, sais e óxidos, suas propriedades e reações. 
        - Reações Químicas: Tipos de reações, balanceamento de equações, reagente limitante, rendimento da reação e cálculo estequiométrico. 
        - Cálculos Químicos: Conceito de mol, número de Avogadro, massa molar, volume molar e cálculos relacionados a reações químicas. 
        - Termoquímica: Estudo do calor envolvido nas reações químicas, entalpia, lei de Hess e aplicações. 
        - Eletroquímica: Estudo da relação entre eletricidade e reações químicas, pilhas, eletrólise e potenciais de eletrodo. 
        - Química Orgânica: Estudo dos compostos de carbono, funções orgânicas, isomeria e reações orgânicas. 
        - Misturas e Separação de Misturas: Tipos de misturas (homogêneas e heterogêneas), métodos de separação (filtração, destilação, decantação, etc.). 
        - Equilíbrio Químico: Conceito de equilíbrio químico, constantes de equilíbrio, deslocamento do equilíbrio. 
        - Química Ambiental: Estudo da poluição ambiental, tratamento de resíduos e impacto das atividades humanas no meio ambiente. 
        - Radioatividade: Desintegração radioativa, tipos de radiação, meia-vida e aplicações da radioatividade.  
        - Pessoas influentes da area
        - Outras Informações

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)

chain_prompt = (
    PromptTemplate.from_template(
        """
        Responda à seguinte pergunta:

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)           


Um RunnableBranch é um tipo especial de executável que permite definir um conjunto de condições e executáveis a serem executados com base na entrada. Ele não oferece nada que você não possa obter em uma função personalizada:

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "física" in x["topic"].lower(), chain_fisica),
    (lambda x: "química" in x["topic"].lower(), chain_quimica),
    (lambda x: "matemática" in x["topic"].lower(), chain_matematica),
    chain_prompt,
)
 

full_chain = {"topic": chain, "query": lambda x: x["query"]} | branch
pretty_print(full_chain.invoke({"query": "Quanto é 2 +2?"}))



In [ ]:
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import PromptTemplate
 

# ela inicial (classificação)
chain = (
    PromptTemplate.from_template(
        """
        Classifique a pergunta do usuário em uma das seguintes categorias:
        - Assuntos Financeiros
        - Suporte Técnico
        - Atualização de Cadastro
        - Outras Informações

        Pergunta: {query}
        Classificação:
        """
    )
    | llm
    | StrOutputParser() 
)

# elos específicos
financial_chain = PromptTemplate.from_template(
    """
    Você é um especialista financeiro.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Financeiro".
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | llm
tech_support_chain = PromptTemplate.from_template(
    """
    Você é um especialista em suporte técnico.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
) | llm
update_registration_chain = PromptTemplate.from_template(
    """
    Você é um representante de atendimento ao cliente.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte de Cadastro".
    Guie o usuário na atualização de suas informações de cadastro.
    Pergunta: {query}
    Resposta:
    """
) | llm
other_info_chain = PromptTemplate.from_template(
    """
    Você é um assistente de informações gerais.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Geral".
    Forneça informações ao usuário sobre sua pergunta.
    Pergunta: {query}
    Resposta:
    """
) | llm
 
 
# Usando a RouterChain
#resposta = router_chain.run("Qual é a segunda Lei de Newton?")
#print(resposta)

In [ ]:
# Função de roteamento
def route(info):
    topic = info["topic"].lower()
    if "financeiro" in topic:
        return financial_chain
    elif "técnico" in topic:
        return tech_support_chain
    elif "atualização" in topic or "cadastro" in topic:
        return update_registration_chain
    else:
        return other_info_chain

In [ ]:
# Exemplos 1 suporte técnico
classification = chain.invoke({"query": "Como faço para redefinir minha senha?"})
print(classification)

In [ ]:
#chama a função rote, passando o topico
response_chain = route({"topic": classification})
print(response_chain)

In [ ]:
#executa o objeto correto
response = response_chain.invoke({"query": "Como faço para redefinir minha senha?"})
pretty_print(response)

In [ ]:
from langchain_core.runnables import RunnableLambda

router_chain = {"topic": chain, "query": lambda x: x["query"]} | RunnableLambda(
    route
)

query = "Qual é a missão da empresa?"
#query = "Como posso pagar uma fatura atrasada?"
#query = "Preciso alterar meu endereço de e-mail."

#query = "Como faço para redefinir minha senha?"

resposta = router_chain.invoke({"query": query})
pretty_print(resposta)

In [ ]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = """Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite que não sabe. 

Aqui vai uma pergunta, seja curto na resposta:
{query}"""

math_template = """Você é um matemático muito bom. 
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor problemas difíceis em suas partes componentes, responder às partes componentes e, em seguida, juntá-las para responder à questão mais ampla.

Aqui está uma pergunta, seja curto na resposta:
{query}"""

prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)
 
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    
    most_similar = prompt_templates[similarity.argmax()]  
 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)


In [ ]:
print(chain.invoke("What's a path integral?"))